In [1]:
from peft import PeftModel, PeftConfig

In [2]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.4 MB/s eta 0:00:00


## Merged 16 bit model

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("manojbaniya/merged16bit")
model = AutoModelForCausalLM.from_pretrained("manojbaniya/merged16bit")

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=

In [5]:
prompt = """
Below is an instruction that describes a task paired with an input that provides further context. Write a response that appriately complete the request.

### instruction:
{instruction}

### question:
{question}

### input:
{context}

### response:
{response}
"""

In [6]:
def get_inputs(query, prompt=prompt):
  instruction = "User le gareko question lai answer deu"
  question = query
  context = ""
  response = ""
  inputs = prompt.format(instruction=instruction, question=question, context=context, response=response)
  inputs = tokenizer(inputs, return_tensors="pt")
  return inputs

In [7]:
inputs = get_inputs("Nepal ko capital city Kathmandu ho vane India ko kaha ho?")
inputs

{'input_ids': tensor([[     2,    108,  33501,    603,    671,  14239,    674,  19306,    476,
           6911,  44600,    675,    671,   3772,    674,   6572,   4024,   4807,
         235265,  15615,    476,   3590,    674, 207546,   2823,   3407,    573,
           3853, 235265,    109,   6176,  14239, 235292,    108,   2224,    709,
          74323,   2034,   2872,  50704,   3448,  55838,    109,   6176,   2872,
         235292,    108, 126672,   5778,   6037,   3413, 116769,   1965, 151145,
           5339,   5778,   5675,    527,   1965, 235336,    109,   6176,   3772,
         235292,    110,   6176,   3590, 235292,    109]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [8]:
outputs = model(inputs["input_ids"])

The 'batch_size' argument of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


In [9]:
logits = outputs.logits[:, -1, :]

In [10]:
preds = logits.softmax(dim=1)

In [11]:
label = preds.argmax(dim=-1)

In [12]:
label

tensor([126672])

In [13]:
tokenizer.decode(label)

'Nepal'

In [14]:
import torch

In [15]:
def generate_tokens(query, max_new_tokens=20):
  inputs = get_inputs(query)
  is_done = False
  tokens = []
  while not is_done:
    with torch.no_grad():
      outputs = model(inputs["input_ids"])
      logits = outputs.logits[:, -1, :]
      preds = logits.softmax(dim=1)
      label = preds.argmax(dim=-1)
      print(tokenizer.decode(label))
      tokens.append(label)
      inputs["input_ids"] = torch.cat([inputs["input_ids"], label.unsqueeze(-1)], dim=-1)
      if label == tokenizer.eos_token_id or len(tokens) >= max_new_tokens:
        is_done = True
  return tokens

In [16]:
generate_tokens("Nepal ko national language lai lai vaninxa?")

Nepal
 ko
 national
 language
 Nepali
 ho
.


<eos>


[tensor([126672]),
 tensor([5778]),
 tensor([4975]),
 tensor([5255]),
 tensor([205184]),
 tensor([1965]),
 tensor([235265]),
 tensor([108]),
 tensor([1])]

In [17]:
generate_tokens("ai vaneko ke ho")

ai
 van
eko
 ke
 ho
,
 ra
kh
na
.


<eos>


[tensor([1515]),
 tensor([2237]),
 tensor([50525]),
 tensor([1708]),
 tensor([1965]),
 tensor([235269]),
 tensor([1051]),
 tensor([12348]),
 tensor([556]),
 tensor([235265]),
 tensor([108]),
 tensor([1])]

In [18]:
generate_tokens("AI ko full form ke ho?")

AI
 ko
 full
 form
 '
Artificial
 Intelligence
'
 ho
.


<eos>


[tensor([11716]),
 tensor([5778]),
 tensor([2247]),
 tensor([1736]),
 tensor([777]),
 tensor([105460]),
 tensor([30515]),
 tensor([235303]),
 tensor([1965]),
 tensor([235265]),
 tensor([108]),
 tensor([1])]